In [15]:
# Gerekli kütüphaneleri import edelim
import os
import glob # Dosya yollarını bulmak için
from tqdm import tqdm # İlerleme çubuğu için
from dotenv import load_dotenv
from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFium2Loader

In [16]:
# .env dosyasındaki environment variable'ları yükle
load_dotenv()

True

In [17]:
# Google API anahtarını kontrol et
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY environment variable not set!")

In [18]:
# Veri klasörünün yolu
DATA_PATH = '../data/' # PDF'lerin bulunduğu klasör

# Vektör veritabanını kaydedeceğimiz yer
DB_FAISS_PATH = 'vectorstore/db_faiss'

In [21]:
# 1. Adım: PDF'leri Yükleme
# DATA_PATH içindeki tüm alt klasörlerdeki PDF'lerin yollarını bul
pdf_files = glob.glob(f"{DATA_PATH}**/*.pdf", recursive=True)
total_files = len(pdf_files)
all_documents = [] # Tüm başarılı dokümanları burada biriktireceğiz

print(f"Toplam {total_files} adet PDF dosyası bulundu. İşlem başlatılıyor...")

# Döngüyü enumerate ile kurarak dosya sayacını da alıyoruz
for i, pdf_path in enumerate(pdf_files):
    # Her döngünün başında hangi dosyayı işlediğimizi yazdırıyoruz
    print(f"[{i+1}/{total_files}] İşleniyor: {pdf_path}")
    try:
        loader = UnstructuredPDFLoader(
            pdf_path,
            strategy="hi_res",
            languages=["tur"]
        )
        documents = loader.load()
        all_documents.extend(documents)
    except Exception as e:
        print(f"    -> !!! UYARI: Bu dosya atlandı. Hata: {e}")
        continue

print(f"\nİşlem tamamlandı. Toplam {len(all_documents)} adet doküman (sayfa) başarıyla yüklendi.")

[22/30] İşleniyor: ../data\2023\THY A.O. YK Faaliyet Raporu Aralık 2023.pdf
[23/30] İşleniyor: ../data\2023\Tupras-2023-Entegre-Faaliyet-Raporu.pdf
[24/30] İşleniyor: ../data\2023\Tupras-Konsolide-SPK-31122023.pdf
[25/30] İşleniyor: ../data\2024\31122024 Akbank Konsolide Finansal Tablolar ve Dipnotlar.pdf
[26/30] İşleniyor: ../data\2024\Akbank Entegre Faaliyet Raporu 2024 - Final.pdf
[27/30] İşleniyor: ../data\2024\THY A.O. Aralık 2024.pdf


Cannot set gray non-stroke color because /'P61' is an invalid float value
Cannot set gray non-stroke color because /'P63' is an invalid float value
Cannot set gray non-stroke color because /'P64' is an invalid float value
Cannot set gray non-stroke color because /'P71' is an invalid float value
Cannot set gray non-stroke color because /'P73' is an invalid float value
Cannot set gray non-stroke color because /'P74' is an invalid float value
Cannot set gray non-stroke color because /'P75' is an invalid float value
Cannot set gray non-stroke color because /'P76' is an invalid float value
Cannot set gray non-stroke color because /'P77' is an invalid float value
Cannot set gray non-stroke color because /'P78' is an invalid float value
Cannot set gray non-stroke color because /'P79' is an invalid float value
Cannot set gray non-stroke color because /'P80' is an invalid float value
Cannot set gray non-stroke color because /'P81' is an invalid float value
Cannot set gray non-stroke color becau

[28/30] İşleniyor: ../data\2024\THY A.O. YK Faaliyet Raporu Aralık 2024.pdf
[29/30] İşleniyor: ../data\2024\tupras-2024-entegre-faaliyet-raporu.pdf


Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is a

[30/30] İşleniyor: ../data\2024\tupras-konsolide-spk-31122024.pdf

İşlem tamamlandı. Toplam 29 adet doküman (sayfa) başarıyla yüklendi.


In [22]:
# 2. Adım: Dokümanları Parçalara Ayırma (Chunking)
# RAG mimarisinde, uzun dokümanları daha küçük ve yönetilebilir parçalara böleriz.
# Bu, arama doğruluğunu artırır.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
docs = text_splitter.split_documents(all_documents)
print(f"Dokümanlar {len(docs)} adet parçaya (chunk) bölündü.")

Dokümanlar 11455 adet parçaya (chunk) bölündü.


In [23]:
# 3. Adım: Gömme (Embedding) Modelini Oluşturma
# Her bir metin parçasını sayısal bir vektöre dönüştürmek için Gemini'nin embedding modelini kullanacağız.
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [24]:
# 4. Adım: Vektör Veritabanı Oluşturma ve Kaydetme
# Tüm metin parçalarını ve onların vektör karşılıklarını FAISS veritabanına kaydediyoruz.
# Bu işlem, veri setinizin büyüklüğüne göre biraz zaman alabilir.
db = FAISS.from_documents(docs, embeddings)
db.save_local(DB_FAISS_PATH)

print("Vektör veritabanı başarıyla oluşturuldu ve 'vectorstore/db_faiss' klasörüne kaydedildi.")

Vektör veritabanı başarıyla oluşturuldu ve 'vectorstore/db_faiss' klasörüne kaydedildi.
